In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from qgsw.utils.sorting import sort_files
from mpl_toolkits.axes_grid1 import make_axes_locatable
from qgsw.run_summary import RunSummary
from matplotlib.axes import Axes

In [ ]:
ROOT = Path(os.path.abspath('')).parent

In [ ]:
folder = ROOT.joinpath("output/g5k/two_layers_baroclinic_100km_long")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))
model = run.configuration.model
steps, files = sort_files(folder.glob(f"{model.prefix}*.npz"), prefix=model.prefix, suffix=".npz")

In [ ]:
%matplotlib tk

plt.ion()

fig = plt.figure(figsize=(12,18))

top_axes: Axes = plt.subplot2grid(fig=fig, shape=(3,2), loc=(0,0))
top_cbar_axes: Axes = make_axes_locatable(top_axes).append_axes("right", size="7%", pad="2%")
bottom_axes: Axes = plt.subplot2grid(fig=fig, shape=(3,2), loc=(0,1))
bottom_cbar_axes: Axes = make_axes_locatable(bottom_axes).append_axes("right", size="7%", pad="2%")
ratio_axes:Axes = plt.subplot2grid(fig=fig, shape=(3,2), loc=(1,0), rowspan=2, colspan=2)
ratio_cbar_axes: Axes = make_axes_locatable(ratio_axes).append_axes("right", size="7%", pad="2%")

for i, file in enumerate(files):
    step = steps[i]
    fig.suptitle(f"Vorticity Ratio Estimation\nTime: {step*run.dt} sec / {steps[-1]*run.dt} sec")
    omega = np.load(file)["omega"]

    top_axes.cla()
    top_axes.set_title("Top Layer Potential Vorticity")
    top_cbar_axes.cla()
    bottom_axes.cla()
    bottom_axes.set_title("Bottom Layer Potential Vorticity")
    bottom_cbar_axes.cla()
    ratio_axes.cla()
    ratio_axes.set_title("Bottom / Top Potential Vorticity Ratio")
    ratio_cbar_axes.cla()

    top: np.ndarray = omega[0,0,...]
    bottom: np.ndarray = omega[0,1,...]

    is_not_zero = np.abs(top) > 0.2

    top_cbar = top_axes.imshow(top, vmin=-np.max(np.abs(top)), vmax=np.max(np.abs(top)))
    fig.colorbar(top_cbar, cax=top_cbar_axes)
    bottom_cbar = bottom_axes.imshow(bottom, vmin=-np.max(np.abs(bottom)), vmax=np.max(np.abs(bottom)))
    fig.colorbar(bottom_cbar, cax=bottom_cbar_axes)

    ratio = np.zeros(top.shape)
    ratio[is_not_zero] = bottom[is_not_zero] / top[is_not_zero]
    ratio_cbar = ratio_axes.imshow(ratio, vmin=-np.max(np.abs(ratio)), vmax=np.max(np.abs(ratio)))
    fig.colorbar(ratio_cbar, cax=ratio_cbar_axes)

    plt.pause(0.01)

plt.ioff()
plt.close()